In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

# データセットがCOCOフォーマットなら以下のコードは次の3行で置き換えることができる。
from detectron2.data.datasets import register_coco_instances

register_coco_instances("fox_falco_train", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train")
register_coco_instances("fox_falco_val", {}, "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val.json", "/home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_val")


1.6.0 True
gcc (Ubuntu 5.5.0-12ubuntu1~16.04) 5.5.0 20171010
Copyright (C) 2015 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



In [3]:
from detectron2.modeling import build_model
import numpy as np
import os, json, cv2, random
import matplotlib.pyplot as plt
import detectron2

from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.utils.visualizer import ColorMode, Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.structures import BoxMode

from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import ColorMode
from detectron2.structures import Boxes

import tqdm
import time

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # falco, fox, smallBF
cfg.MODEL.WEIGHTS = "/home/tappun/wip/output/model_final.pth"
cfg.MODEL.DEVICE='cuda:3'

cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.65
cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.65
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.65
cfg.MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH = 0.65

model = build_model(cfg)

predictor = DefaultPredictor(cfg)
# dataset_dicts = DatasetCatalog.get("fox_falco_val_2")

#ワンチャンGPUで早くなるかも


# video = cv2.VideoCapture('/home/tappun/short_fox_falco.mp4')
video = cv2.VideoCapture('/home/tappun/ssbu_dataset/movie/fox_falco_smallBF1.mp4')

print(video)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize video writer
video_writer = cv2.VideoWriter('totemonagai2.mp4', fourcc=cv2.VideoWriter_fourcc(*"mp4v"), fps=float(frames_per_second), frameSize=(width, height), isColor=True)

v = VideoVisualizer(MetadataCatalog.get("fox_falco_val_2").set(thing_classes=["Falco", "Fox", "SmallBattleField"]),ColorMode.IMAGE)


def runOnVideo(video, maxFrames):
    """ Runs the predictor on every frame in the video (unless maxFrames is given),
    and returns the frame with the predictions drawn.
    """

    readFrames = 0
    while True:
        
        hasFrame, frame = video.read()
        
        if not hasFrame:
            print("動画の読み込みに失敗")
            break

        # Get prediction results for this frame
        outputs = predictor(frame)

        # Make sure the frame is colored
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

        # Draw a visualization of the predictions using the video visualizer
        visualization = v.draw_instance_predictions(frame, outputs["instances"].to("cpu"))

        # Convert Matplotlib RGB format to OpenCV BGR format
        visualization = cv2.cvtColor(visualization.get_image(), cv2.COLOR_RGB2BGR)

        yield visualization

        readFrames += 1
        if readFrames > maxFrames:
            print("obobo")
            break
            
# metadata=MetadataCatalog.get("fox_falco_val_2").set(thing_classes=["Falco", "Fox", "PokemonStadium2"]),

# Create a cut-off for debugging
num_frames = 3600

# Enumerate the frames of the video
for visualization in tqdm.tqdm(runOnVideo(video, num_frames), total=num_frames):
    # Write test image
    cv2.imwrite('POSE detectron2.png', visualization)
    # Write to video file
    video_writer.write(visualization)

# Release resources
video.release()
video_writer.release()
# cv2.destroyAllWindows()

  0%|          | 0/3600 [00:00<?, ?it/s]

<VideoCapture 0x7f391c435c10>


3601it [13:54,  4.31it/s]                          

obobo


In [2]:

def train_dataset(dataset_name, dataset_name_val):
    cfg = get_cfg()
    cfg.MODEL.DEVICE='cuda:3'
#     cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.DATASETS.TRAIN = (dataset_name,)
    cfg.DATASETS.TEST = (dataset_name_val,)
    cfg.DATALOADER.NUM_WORKERS = 2
    #cfg.MODEL.WEIGHTS = "detectron2://COCO-Detection/faster_rcnn_R_101_FPN_3x/137851257/model_final_f6e8b1.pkl"
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_C4_1x.yaml")
    
    #https://github.com/facebookresearch/detectron2/blob/master/configs/COCO-Detection/faster_rcnn_R_50_C4_1x.yaml
#     cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
    cfg.SOLVER.IMS_PER_BATCH = 2
    cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
    cfg.SOLVER.MAX_ITER = 1000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # faster, and good enough for this toy dataset (default: 512)
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)
    cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.7

    os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
    trainer = DefaultTrainer(cfg) 
    trainer.resume_or_load(resume=False)
    
    print(dataset_name)
    
    trainer.train()

    
if __name__ == '__main__':
    # Prepare the data set
    dataset_name = "fox_falco_train"
    dataset_name_val = "fox_falco_val"
#     dicts_name   = "balloon/train"
#     for d in ["train", "val"]:
#         DatasetCatalog.register("balloon_" + d, lambda d=d: get_balloon_dicts("balloon/" + d))
#         MetadataCatalog.get("balloon_" + d).set(thing_classes=["balloon"])
#     balloon_metadata = MetadataCatalog.get(dataset_name)
#     # Verify the data set
#     verify_dataset(dicts_name)

    # train
    train_dataset(dataset_name, dataset_name_val)

[12/10 22:24:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
 

WARNING [12/10 22:24:39 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/10 22:24:39 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/10 22:24:39 d2.data.build]: Removed 0 images with no usable annotations. 448 images left.
[12/10 22:24:39 d2.data.build]: Distribution of instances among all 3 categories:
|  category  | #instances   |  category  | #instances   |   category    | #instances   |
|:----------:|:-------------|:----------:|:-------------|:-------------:|:-------------|
|   Falco    | 442          |    Fox     | 421          | SmallBattle.. | 416          |
|            |              |            |              |               |              |
|   total    | 1279         |            |              |               |              |
[12/10 22:24:39 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeSho

model_final_721ade.pkl: 136MB [00:12, 10.5MB/s]                               
Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 2048) in the checkpoint but (4, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 2048) in the checkpoint but (12, 2048) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.


fox_falco_train
[12/10 22:24:52 d2.engine.train_loop]: Starting training from iteration 0


/home/tappun/.pyenv/versions/3.6.8/lib/python3.6/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:217: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  num_fg = fg_inds.nonzero().numel()


[12/10 22:25:18 d2.utils.events]:  eta: 0:19:12  iter: 19  total_loss: 2.298  loss_cls: 1.329  loss_box_reg: 0.4149  loss_rpn_cls: 0.1989  loss_rpn_loc: 0.3387  time: 1.1729  data_time: 0.0117  lr: 4.9953e-06  max_mem: 0M
[12/10 22:25:42 d2.utils.events]:  eta: 0:18:52  iter: 39  total_loss: 2.105  loss_cls: 1.224  loss_box_reg: 0.4255  loss_rpn_cls: 0.1932  loss_rpn_loc: 0.2162  time: 1.1735  data_time: 0.0040  lr: 9.9902e-06  max_mem: 0M
[12/10 22:26:06 d2.utils.events]:  eta: 0:18:34  iter: 59  total_loss: 1.92  loss_cls: 1.058  loss_box_reg: 0.4295  loss_rpn_cls: 0.1862  loss_rpn_loc: 0.2603  time: 1.1769  data_time: 0.0038  lr: 1.4985e-05  max_mem: 0M
[12/10 22:26:30 d2.utils.events]:  eta: 0:18:16  iter: 79  total_loss: 1.742  loss_cls: 0.8568  loss_box_reg: 0.4287  loss_rpn_cls: 0.1904  loss_rpn_loc: 0.2818  time: 1.1804  data_time: 0.0040  lr: 1.998e-05  max_mem: 0M
[12/10 22:26:53 d2.utils.events]:  eta: 0:17:48  iter: 99  total_loss: 1.502  loss_cls: 0.663  loss_box_reg: 0.40

[12/10 22:39:45 d2.utils.events]:  eta: 0:05:13  iter: 739  total_loss: 0.8341  loss_cls: 0.1913  loss_box_reg: 0.407  loss_rpn_cls: 0.04071  loss_rpn_loc: 0.1757  time: 1.2016  data_time: 0.0037  lr: 0.00018482  max_mem: 0M
[12/10 22:40:14 d2.utils.events]:  eta: 0:04:50  iter: 759  total_loss: 0.7712  loss_cls: 0.1531  loss_box_reg: 0.3891  loss_rpn_cls: 0.0409  loss_rpn_loc: 0.1619  time: 1.2069  data_time: 0.0035  lr: 0.00018981  max_mem: 0M
[12/10 22:40:41 d2.utils.events]:  eta: 0:04:26  iter: 779  total_loss: 0.7252  loss_cls: 0.1421  loss_box_reg: 0.3815  loss_rpn_cls: 0.02896  loss_rpn_loc: 0.1584  time: 1.2114  data_time: 0.0035  lr: 0.00019481  max_mem: 0M
[12/10 22:41:07 d2.utils.events]:  eta: 0:04:02  iter: 799  total_loss: 0.6875  loss_cls: 0.1421  loss_box_reg: 0.3307  loss_rpn_cls: 0.03057  loss_rpn_loc: 0.1624  time: 1.2128  data_time: 0.0038  lr: 0.0001998  max_mem: 0M
[12/10 22:41:31 d2.utils.events]:  eta: 0:03:38  iter: 819  total_loss: 0.7797  loss_cls: 0.1468  l

In [3]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

dataset_name = "fox_falco_train"
dataset_name_val = "fox_falco_val"

cfg = get_cfg()
cfg.MODEL.DEVICE='cuda:3'
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = (dataset_name,)
cfg.DATASETS.TEST = (dataset_name_val,)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000   # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256  # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # only has one class (ballon)
cfg.MODEL.ROI_HEADS.SCORE_THRESHOLD_TEST = 0.7

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)

evaluator = COCOEvaluator(dataset_name, ("bbox", ), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, dataset_name)
inference_on_dataset(trainer.model, val_loader, evaluator)

[12/10 23:06:44 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

WARNING [12/10 23:06:44 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/10 23:06:44 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/10 23:06:44 d2.data.build]: Removed 0 images with no usable annotations. 448 images left.
[12/10 23:06:44 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[12/10 23:06:44 d2.data.build]: Using training sampler TrainingSampler
[12/10 23:06:44 d2.data.common]: Serializing 448 elements to byte tensors and concatenating them all ...
[12/10 23:06:44 d2.data.common]: Serialized dataset takes 0.24 MiB


Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (3, 256, 1, 1) 

WARNING [12/10 23:06:45 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[12/10 23:06:45 d2.data.datasets.coco]: Loaded 448 images in COCO format from /home/tappun/ssbu_dataset/20201126/fox_falco_smallBF_train.json
[12/10 23:06:45 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/10 23:06:45 d2.data.common]: Serializing 448 elements to byte tensors and concatenating them all ...
[12/10 23:06:45 d2.data.common]: Serialized dataset takes 0.24 MiB
[12/10 23:06:45 d2.evaluation.evaluator]: Start inference on 448 images
[12/10 23:06:50 d2.evaluation.evaluator]: Inference done 11/448. 0.1911 s / img. ETA=0:02:58
[12/10 23:06:55 d2.evaluation.evaluator]: Inference done 23/448. 0.2275 s / img. ETA=0:03:06
[12/10 23:07:00 d2.evaluation.evaluator]: Inference done 35/448. 0.2229 s / img. ETA=0:02:59
[12/10 23:07:05 d2

OrderedDict([('bbox',
              {'AP': 0.01286946355094265,
               'AP50': 0.06786623391716579,
               'AP75': 0.0013017121965733054,
               'APs': nan,
               'APm': 0.009210646667438307,
               'APl': 0.038797082816075634,
               'AP-Falco': 0.01023696579217589,
               'AP-Fox': 0.014551115597108668,
               'AP-SmallBattleField': 0.013820309263543384})])

In [2]:
detectron2.__version__

'0.3'

In [ ]:
!pip uninstall detectron2

Found existing installation: detectron2 0.2.1+cu102
Uninstalling detectron2-0.2.1+cu102:
  Would remove:
    /home/tappun/.pyenv/versions/3.6.8/lib/python3.6/site-packages/detectron2-0.2.1+cu102.dist-info/*
    /home/tappun/.pyenv/versions/3.6.8/lib/python3.6/site-packages/detectron2/*
Proceed (y/n)? 